In [7]:
import sys
!{sys.executable} -m pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 44.9 MB/s eta 0:00:00 0:00:01


# Part 1 (Relational)

### Load all the data into a duckdb database

In [8]:
import duckdb

In [9]:
# connect to a duckDB database file
con = duckdb.connect('mimic3.duckdb')

In [10]:
csv_files = [
    'ADMISSIONS.csv',
    'ICUSTAYS.csv',
    'PATIENTS.csv',
    'PROCEDURES_ICD.csv',
    'D_ICD_PROCEDURES.csv',
    'DRGCODES.csv',
    'PRESCRIPTIONS.csv'
]

In [12]:
# create a duckdb table for each csv file
for fname in csv_files:
    table_name = fname.split('.')[0].lower()
    con.execute(f"""
        CREATE OR REPLACE TABLE {table_name} AS
        SELECT * 
        FROM read_csv_auto('{fname}');
    """)
print(con.execute("SHOW TABLES").fetchall())

[('admissions',), ('d_icd_procedures',), ('drgcodes',), ('icustays',), ('patients',), ('prescriptions',), ('procedures_icd',)]


### EDA

In [13]:
# get columns names/types
con.execute("PRAGMA table_info('prescriptions')").df()

,cid,name,type,notnull,dflt_value,pk
0,0,row_id,BIGINT,False,None,False
1,1,subject_id,BIGINT,False,None,False
2,2,hadm_id,BIGINT,False,None,False
3,3,icustay_id,BIGINT,False,None,False
4,4,startdate,TIMESTAMP,False,None,False
5,5,enddate,TIMESTAMP,False,None,False
6,6,drug_type,VARCHAR,False,None,False
7,7,drug,VARCHAR,False,None,False
8,8,drug_name_poe,VARCHAR,False,None,False
9,9,drug_name_generic,VARCHAR,False,None,False


In [14]:
con.execute("SELECT * FROM prescriptions LIMIT 5").df()

,row_id,subject_id,hadm_id,icustay_id,startdate,enddate,drug_type,drug,drug_name_poe,drug_name_generic,formulary_drug_cd,gsn,ndc,prod_strength,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,route
0,32600,42458,159647,<NA>,2146-07-21,2146-07-22,MAIN,Pneumococcal Vac Polyvalent,Pneumococcal Vac Polyvalent,PNEUMOcoccal Vac Polyvalent,PNEU25I,048548,00006494300,25mcg/0.5mL Vial,0.5,mL,1,VIAL,IM
1,32601,42458,159647,<NA>,2146-07-21,2146-07-22,MAIN,Bisacodyl,Bisacodyl,Bisacodyl,BISA5,002947,00536338101,5 mg Tab,10,mg,2,TAB,PO
2,32602,42458,159647,<NA>,2146-07-21,2146-07-22,MAIN,Bisacodyl,Bisacodyl,Bisacodyl (Rectal),BISA10R,002944,00574705050,10mg Suppository,10,mg,1,SUPP,PR
3,32603,42458,159647,<NA>,2146-07-21,2146-07-22,MAIN,Senna,Senna,Senna,SENN187,019964,00904516561,1 Tablet,1,TAB,1,TAB,PO
4,32604,42458,159647,<NA>,2146-07-21,2146-07-21,MAIN,Docusate Sodium (Liquid),Docusate Sodium (Liquid),Docusate Sodium (Liquid),DOCU100L,003017,00121054410,100mg UD Cup,100,mg,1,UDCUP,PO


In [15]:
# admissions names/types
con.execute("PRAGMA table_info('admissions')").df()

,cid,name,type,notnull,dflt_value,pk
0,0,row_id,BIGINT,False,None,False
1,1,subject_id,BIGINT,False,None,False
2,2,hadm_id,BIGINT,False,None,False
3,3,admittime,TIMESTAMP,False,None,False
4,4,dischtime,TIMESTAMP,False,None,False
5,5,deathtime,TIMESTAMP,False,None,False
6,6,admission_type,VARCHAR,False,None,False
7,7,admission_location,VARCHAR,False,None,False
8,8,discharge_location,VARCHAR,False,None,False
9,9,insurance,VARCHAR,False,None,False


In [16]:
con.execute("SELECT hadm_id, subject_id, ethnicity FROM admissions LIMIT 5").df()

,hadm_id,subject_id,ethnicity
0,142345,10006,BLACK/AFRICAN AMERICAN
1,105331,10011,UNKNOWN/NOT SPECIFIED
2,165520,10013,UNKNOWN/NOT SPECIFIED
3,199207,10017,WHITE
4,177759,10019,WHITE


In [17]:
con.execute("SELECT DISTINCT drug_type FROM prescriptions").df()

,drug_type
0,ADDITIVE
1,MAIN
2,BASE


In [39]:
con.execute("PRAGMA table_info('patients')").df()

,cid,name,type,notnull,dflt_value,pk
0,0,row_id,BIGINT,False,None,False
1,1,subject_id,BIGINT,False,None,False
2,2,gender,VARCHAR,False,None,False
3,3,dob,TIMESTAMP,False,None,False
4,4,dod,TIMESTAMP,False,None,False
5,5,dod_hosp,TIMESTAMP,False,None,False
6,6,dod_ssn,TIMESTAMP,False,None,False
7,7,expire_flag,BIGINT,False,None,False


In [40]:
con.execute("SELECT subject_id, dob, gender FROM patients LIMIT 5").df()

,subject_id,dob,gender
0,10006,2094-03-05,F
1,10011,2090-06-05,F
2,10013,2038-09-03,F
3,10017,2075-09-21,F
4,10019,2114-06-20,M


In [41]:
con.execute("PRAGMA table_info('d_icd_procedures')").df()

,cid,name,type,notnull,dflt_value,pk
0,0,row_id,BIGINT,False,None,False
1,1,icd9_code,VARCHAR,False,None,False
2,2,short_title,VARCHAR,False,None,False
3,3,long_title,VARCHAR,False,None,False


In [42]:
con.execute("""
  SELECT icd9_code,
         short_title,
         long_title
  FROM d_icd_procedures
  LIMIT 5
""").df()

,icd9_code,short_title,long_title
0,1423,Chorioret les xenon coag,Destruction of chorioretinal lesion by xenon a...
1,1424,Chorioret les laser coag,Destruction of chorioretinal lesion by laser p...
2,1425,Chorioret les p/coag NOS,Destruction of chorioretinal lesion by photoco...
3,1426,Chorioret les radiother,Destruction of chorioretinal lesion by radiati...
4,1427,Chorioret les rad implan,Destruction of chorioretinal lesion by implant...


In [43]:
con.execute("PRAGMA table_info('procedures_icd')").df()

,cid,name,type,notnull,dflt_value,pk
0,0,row_id,BIGINT,False,None,False
1,1,subject_id,BIGINT,False,None,False
2,2,hadm_id,BIGINT,False,None,False
3,3,seq_num,BIGINT,False,None,False
4,4,icd9_code,VARCHAR,False,None,False


In [44]:
con.execute("SELECT * FROM procedures_icd LIMIT 5").df()

,row_id,subject_id,hadm_id,seq_num,icd9_code
0,3994,10114,167957,1,3605
1,3995,10114,167957,2,3722
2,3996,10114,167957,3,8856
3,3997,10114,167957,4,9920
4,3998,10114,167957,5,9671


In [47]:
con.execute("PRAGMA table_info('icustays')").df()

,cid,name,type,notnull,dflt_value,pk
0,0,row_id,BIGINT,False,None,False
1,1,subject_id,BIGINT,False,None,False
2,2,hadm_id,BIGINT,False,None,False
3,3,icustay_id,BIGINT,False,None,False
4,4,dbsource,VARCHAR,False,None,False
5,5,first_careunit,VARCHAR,False,None,False
6,6,last_careunit,VARCHAR,False,None,False
7,7,first_wardid,BIGINT,False,None,False
8,8,last_wardid,BIGINT,False,None,False
9,9,intime,TIMESTAMP,False,None,False


In [48]:
con.execute("""
  SELECT icustay_id, subject_id, hadm_id, intime, outtime
  FROM icustays
  LIMIT 5
""").df()

,icustay_id,subject_id,hadm_id,intime,outtime
0,206504,10006,142345,2164-10-23 21:10:15,2164-10-25 12:21:07
1,232110,10011,105331,2126-08-14 22:34:00,2126-08-28 18:59:00
2,264446,10013,165520,2125-10-04 23:38:00,2125-10-07 15:13:52
3,204881,10017,199207,2149-05-29 18:52:29,2149-05-31 22:19:17
4,228977,10019,177759,2163-05-14 20:43:56,2163-05-16 03:47:04


## Analysis Questions

### 1) Create a summary of type of drugs and their total amount used by ethnicity. Report the top usage in each ethnicity group. You may have to make certain assumptions in calculating their total amount.

In [45]:
query_drugs = """
WITH usage AS (
  SELECT
    a.ethnicity,
    p.drug_type,
    SUM(
      TRY_CAST(p.dose_val_rx AS DOUBLE) *
      TRY_CAST(p.form_val_disp AS DOUBLE)
    ) AS total_amount
  FROM prescriptions p
  JOIN admissions a
    ON p.hadm_id = a.hadm_id
  WHERE
    TRY_CAST(p.dose_val_rx AS DOUBLE) IS NOT NULL
    AND TRY_CAST(p.form_val_disp AS DOUBLE) IS NOT NULL
  GROUP BY a.ethnicity, p.drug_type
)
SELECT
  ethnicity,
  drug_type,
  total_amount
FROM (
  SELECT
    *,
    ROW_NUMBER() OVER (
      PARTITION BY ethnicity
      ORDER BY total_amount DESC
    ) AS rn
  FROM usage
)
WHERE rn = 1
ORDER BY ethnicity;
"""
result = con.execute(query_drugs).df()
result.head(10)


,ethnicity,drug_type,total_amount
0,AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGN...,BASE,4552659.4
1,ASIAN,BASE,12232001.0
2,BLACK/AFRICAN AMERICAN,BASE,13727971.0
3,HISPANIC OR LATINO,BASE,10490509.0
4,HISPANIC/LATINO - PUERTO RICAN,BASE,39616673.0
5,OTHER,BASE,2227226.0
6,UNABLE TO OBTAIN,BASE,6206703.0
7,UNKNOWN/NOT SPECIFIED,BASE,14886852.0
8,WHITE,BASE,150928090.6


A) shown above

B) The SQL query:
- joins prescriptions to admissions on hadm_id to access ethnicity
- uses TRY_CAST to remove null or NaN values for doses/dispensed values
- calculates the total amount using the formula dose_val_rx * form_val_disp
- groups by ethnicity and drug_type to sum total amounts
- ranks the drug types by total amount for each ethnicty and identifies the most-used drug

C) shown above

D) The findings show that in every ethnicity, BASE is the drug type with the highest usage. This suggests that BASE drugs are given in larger volume than MAIN and ADDITIVE drugs -- a good next step would to be find out how many "doses" of each drug type each ethnicity consumes, to account for the difference in unit type (mg tablets, mL cups, etc.)

### 2) Create a summary of procedures performed on patients by age groups (<=19, 20-49, 50-79, >80). Report the top three procedures, along with the name of the procedures, performed in each age group.

In [46]:
query_procedures = """
WITH proc_age AS (
  SELECT
    pr.icd9_code,
    -- compute age at admission in years
    date_diff('year', pa.dob, ad.admittime) AS age
  FROM procedures_icd pr
  JOIN admissions ad
    ON pr.hadm_id = ad.hadm_id
  JOIN patients pa
    ON pr.subject_id = pa.subject_id
),
buckets AS (
  SELECT
    icd9_code,
    CASE
      WHEN age <= 19 THEN '<=19'
      WHEN age BETWEEN 20 AND 49 THEN '20-49'
      WHEN age BETWEEN 50 AND 79 THEN '50-79'
      ELSE '>80'
    END AS age_group
  FROM proc_age
),
counts AS (
  SELECT
    b.age_group,
    b.icd9_code,
    COUNT(*) AS proc_count
  FROM buckets b
  GROUP BY b.age_group, b.icd9_code
),
with_names AS (
  SELECT
    c.age_group,
    c.icd9_code,
    d.short_title,
    d.long_title,
    c.proc_count
  FROM counts c
  JOIN d_icd_procedures d
    ON c.icd9_code = d.icd9_code
),
ranked AS (
  SELECT
    *,
    ROW_NUMBER() OVER (
      PARTITION BY age_group
      ORDER BY proc_count DESC
    ) AS rn
  FROM with_names
)
SELECT
  age_group,
  icd9_code,
  short_title,
  long_title,
  proc_count
FROM ranked
WHERE rn <= 3
ORDER BY age_group, proc_count DESC;
"""
top_three = con.execute(query_procedures).df()
top_three

,age_group,icd9_code,short_title,long_title,proc_count
0,20-49,3893,Venous cath NEC,"Venous catheterization, not elsewhere classified",9
1,20-49,966,Entral infus nutrit sub,Enteral infusion of concentrated nutritional s...,7
2,20-49,9604,Insert endotracheal tube,Insertion of endotracheal tube,6
3,50-79,3893,Venous cath NEC,"Venous catheterization, not elsewhere classified",25
4,50-79,966,Entral infus nutrit sub,Enteral infusion of concentrated nutritional s...,22
5,50-79,9904,Packed cell transfusion,Transfusion of packed cells,13
6,<=19,3893,Venous cath NEC,"Venous catheterization, not elsewhere classified",2
7,<=19,0353,Vertebral fx repair,Repair of vertebral fracture,1
8,<=19,387,Interruption vena cava,Interruption of the vena cava,1
9,>80,3893,Venous cath NEC,"Venous catheterization, not elsewhere classified",20


A) shown above

B) the SQL query:
- computes each patient's age (at the time they were admitted) with proc_age
- assigns each record to an age group with buckets
- counts how many times each icd9_code occurs in each group
- ranks the most frequent procedures in each age group
- selects the top three most frequent procedures in each age group

C) shown above

D) the top three procedures in each age group are:

Age <= 19:
1. Venous cath NEC (2)
2. Vertebral fx repair (1)
3. Interruption vena cava (1)

Age 20-49:
1. Venous cath NEC (9)
2. Entral infus nutrit sub (7)
3. Insert endotracheal tube	(6)

Age 50-79:
1. Venous cath NEC (25)
2. Entral infus nutrit sub (22)
3. Packed cell transfusion (13)

Age > 80:
1. Venous cath NEC (20)
2. Packed cell transfusion (13)
3. Insert endotracheal tube	(8)

In each age group, Venous cath NEC (venous catherization) is the most frequent procedure, demonstrating the widespread use of IV treatment

### 3) How long do patients stay in the ICU? Is there a difference in the ICU length of stay among gender or ethnicity?

#### Length of stay summary

In [49]:
los_summary = con.execute("""
SELECT
  COUNT(*)       AS total_stays,
  ROUND(AVG(los), 2) AS avg_los_days,
  ROUND(MIN(los), 2) AS min_los_days,
  ROUND(MAX(los), 2) AS max_los_days
FROM icustays;
""").df()
los_summary

,total_stays,avg_los_days,min_los_days,max_los_days
0,136,4.45,0.11,35.41


- COUNT(*) returns total number of ICU stays
- AVG(los) returns the average length of stay in days
- MIN(los) and MAX(los) return the extremes

Length of stay by gender

In [50]:
los_by_gender = con.execute("""
SELECT
  pa.gender,
  COUNT(*)           AS n_stays,
  ROUND(AVG(ic.los), 2) AS avg_los_days,
  ROUND(MIN(ic.los), 2) AS min_los_days,
  ROUND(MAX(ic.los), 2) AS max_los_days
FROM icustays ic
JOIN patients pa
  ON ic.subject_id = pa.subject_id
GROUP BY pa.gender;
""").df()
los_by_gender

,gender,n_stays,avg_los_days,min_los_days,max_los_days
0,F,63,5.54,0.19,35.41
1,M,73,3.51,0.11,21.41


- we join icustays and patients to access gender
- we compute the same summary statistics for each gender

Length of stay by ethnicity

In [54]:
los_by_ethnicity = con.execute("""
SELECT
  ad.ethnicity,
  COUNT(*)           AS n_stays,
  ROUND(AVG(ic.los), 2) AS avg_los_days,
  ROUND(MIN(ic.los), 2) AS min_los_days,
  ROUND(MAX(ic.los), 2) AS max_los_days
FROM icustays ic
JOIN admissions ad
  ON ic.hadm_id = ad.hadm_id
GROUP BY ad.ethnicity
ORDER BY avg_los_days DESC;
""").df()
los_by_ethnicity

,ethnicity,n_stays,avg_los_days,min_los_days,max_los_days
0,UNABLE TO OBTAIN,1,13.36,13.36,13.36
1,AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGN...,2,11.34,1.26,21.41
2,BLACK/AFRICAN AMERICAN,7,7.68,0.86,31.12
3,HISPANIC OR LATINO,3,7.46,3.55,15.06
4,UNKNOWN/NOT SPECIFIED,11,4.93,1.64,15.04
5,WHITE,92,4.13,0.19,35.41
6,ASIAN,2,3.89,0.66,7.12
7,HISPANIC/LATINO - PUERTO RICAN,15,3.24,0.76,10.86
8,OTHER,3,0.93,0.11,1.91


- we join icustays and admissions to access ethnicity
- we calculate the same summary statistics for each ethnicity

A) shown above

B) shown above

C) shown above

D) Overall, there were 136 total stays, with an average length of stay of 4.45 days, ranging from 0.11 days to 35.41 days. On average, female patients stayed in the ICU roughly 2 days longer than male patients (5.54 vs 3.51 days). Smaller ethnicity groups like American Indian/Alaska Native and "UNABLE TO OBTAIN" showed the highest average length of stays (11.34 - 13.36 days) but didn't have a lot of stays. Black/African American (7.68 days) and Hispanic or Latino (7.46 days) patients had longer average stays than White (4.13 days) and Asian (3.89 days) patients.

# Part 2 (Non-relational)

## Lab 4 portion

In [55]:
%pip install cassandra-sigv4

  Using cached cassandra_sigv4-4.0.2-py2.py3-none-any.whl.metadata (5.9 kB)
  Using cached geomet-0.2.1.post1-py3-none-any.whl.metadata (1.0 kB)
Using cached cassandra_sigv4-4.0.2-py2.py3-none-any.whl (9.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 45.2 MB/s eta 0:00:00 0:00:01
Using cached geomet-0.2.1.post1-py3-none-any.whl (18 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.1
    Uninstalling urllib3-2.2.1:
      Successfully uninstalled urllib3-2.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-macos 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.12.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [67]:
from cassandra.cluster import Cluster
from ssl import SSLContext, PROTOCOL_TLSv1_2, CERT_REQUIRED
from cassandra_sigv4.auth import SigV4AuthProvider
import boto3

# ssl setup
ssl_context = SSLContext(PROTOCOL_TLSv1_2)
ssl_context.load_verify_locations('sf-class2-root.crt')  # change your file path for locating the certificate
ssl_context.verify_mode = CERT_REQUIRED

# boto3 session setup
boto_session = boto3.Session(
    profile_name="549787090008_mse-tl-dataeng300-EMR",
    region_name="us-east-2"
)

In [68]:
# authorization setup with SigV4
auth_provider = SigV4AuthProvider(boto_session)

In [69]:
#cluster setup 
cluster = Cluster(['cassandra.us-east-2.amazonaws.com'], 
                  ssl_context=ssl_context, 
                  auth_provider=auth_provider, 
                  port=9142)  # TLS only communicates on port 9142

In [70]:
# establishing connection to Keyspace
session = cluster.connect()

In [71]:
# Insert any CQL queries between .connect() and .shutdown()

# For example, show all keyspaces created
r = session.execute('''
    SELECT * FROM system_schema.keyspaces;
    ''')
print(r.current_rows)

[Row(keyspace_name='system_schema', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '3')])), Row(keyspace_name='system_schema_mcs', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '3')])), Row(keyspace_name='system', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '3')])), Row(keyspace_name='system_multiregion_info', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '3')])), Row(keyspace_name='aet7207_hw2', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '3')])), Row(keyspace_name='atb2199_ks', durable_writes=True, replication=OrderedMapSeri

In [72]:
# For example, create a keyspace for HW2
r = session.execute('''
    CREATE KEYSPACE IF NOT EXISTS de300_demo 
    WITH replication = {'class': 'SingleRegionStrategy'};
    ''')
print(r.current_rows)

[]


In [73]:
from cassandra.cluster import ExecutionProfile, EXEC_PROFILE_DEFAULT
from cassandra import ConsistencyLevel


# Define execution profile with LOCAL_QUORUM
execution_profile = ExecutionProfile(
    consistency_level=ConsistencyLevel.LOCAL_QUORUM
)

# Cluster setup with correct profile
cluster = Cluster(
    ['cassandra.us-east-2.amazonaws.com'],
    ssl_context=ssl_context,
    auth_provider=auth_provider,
    port=9142,
    execution_profiles={EXEC_PROFILE_DEFAULT: execution_profile}
)

# establishing connection to Keyspace
session = cluster.connect()
session.set_keyspace('de300_demo')  # Replace with your keyspace

In [74]:
session.execute("""
CREATE TABLE IF NOT EXISTS github (
    id UUID PRIMARY KEY,
    name TEXT,
    username TEXT
)
""")

In [75]:
import uuid

session.execute("""
    INSERT INTO github (id, name, username)
    VALUES (%s, %s, %s)
""", (uuid.uuid4(), "Ian Evensen", "iane884"))

In [90]:
import csv

rows = session.execute("SELECT * FROM github")
with open("github.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["id", "name", "username"])
    for row in rows:
        writer.writerow([row.id, row.name, row.username])

In [92]:
!mv github.csv DE300_Labs/lab4/

In [95]:
%%bash
cd DE300_Labs
git add lab4/github.csv
git commit -m "lab4 - export table"
git push

[main 28e92d4] lab4 - export table
 1 file changed, 45 insertions(+)
 create mode 100644 lab4/github.csv


To https://github.com/iane884/DE300_Labs.git
   b97521f..28e92d4  main -> main


In [96]:
session.shutdown()

# Part 2 (Non-relational)

No copies of the AWS crendentials file is stored on any publicly accessible location, nor is the file in any way shared with anyone outside of DATA_ENG 300 (Spring 2025)

Ian Evensen

In [98]:
session = cluster.connect()
session.set_keyspace('homework2')

## Analysis Question 1

In [99]:
session.execute("""
CREATE TABLE IF NOT EXISTS drug_usage_by_eth (
    ethnicity     text,
    drug_type     text,
    total_amount  double,
    PRIMARY KEY ((ethnicity), total_amount, drug_type)
) WITH CLUSTERING ORDER BY (total_amount DESC);
""")

In [101]:
rows = session.execute("""
  SELECT table_name
    FROM system_schema.tables
   WHERE keyspace_name='homework2';
""")
print([r.table_name for r in rows])

['drug_counts_by_ethnicity', 'drug_usage_by_eth', 'github', 'icu_stay_records', 'prescriptions_by_ethnicity', 'procedures_by_age_group', 'top_drug_by_ethnicity', 'top_drug_summary', 'top_procedures_by_age', 'top_procedures_by_age_rank']


In [102]:
query_drugs = """
WITH usage AS (
  SELECT
    a.ethnicity,
    p.drug_type,
    SUM(
      TRY_CAST(p.dose_val_rx AS DOUBLE) *
      TRY_CAST(p.form_val_disp AS DOUBLE)
    ) AS total_amount
  FROM prescriptions p
  JOIN admissions a ON p.hadm_id = a.hadm_id
  WHERE TRY_CAST(p.dose_val_rx AS DOUBLE) IS NOT NULL
    AND TRY_CAST(p.form_val_disp AS DOUBLE) IS NOT NULL
  GROUP BY a.ethnicity, p.drug_type
)
SELECT ethnicity, drug_type, total_amount
FROM (
  SELECT *,
    ROW_NUMBER() OVER (
      PARTITION BY ethnicity
      ORDER BY total_amount DESC
    ) AS rn
  FROM usage
)
WHERE rn = 1
ORDER BY ethnicity;
"""

df_drugs = con.execute(query_drugs).df()

insert_drug = session.prepare("""
  INSERT INTO drug_usage_by_eth (ethnicity, drug_type, total_amount)
  VALUES (?, ?, ?)
""")
for row in df_drugs.itertuples(index=False):
    session.execute(insert_drug, (row.ethnicity, row.drug_type, float(row.total_amount)))

In [122]:
rows = session.execute("SELECT ethnicity, drug_type, total_amount FROM drug_usage_by_eth LIMIT 10")
for r in rows:
    print(r)

Row(ethnicity='OTHER', drug_type='BASE', total_amount=2227226.0)
Row(ethnicity='BLACK/AFRICAN AMERICAN', drug_type='BASE', total_amount=13727971.0)
Row(ethnicity='WHITE', drug_type='BASE', total_amount=150928090.60000002)
Row(ethnicity='ASIAN', drug_type='BASE', total_amount=12232001.0)
Row(ethnicity='HISPANIC/LATINO - PUERTO RICAN', drug_type='BASE', total_amount=39616673.0)
Row(ethnicity='UNKNOWN/NOT SPECIFIED', drug_type='BASE', total_amount=14886852.0)
Row(ethnicity='UNABLE TO OBTAIN', drug_type='BASE', total_amount=6206703.0)
Row(ethnicity='AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE', drug_type='BASE', total_amount=4552659.4)
Row(ethnicity='HISPANIC OR LATINO', drug_type='BASE', total_amount=10490509.0)


## Analysis Question 2

In [108]:
session.execute("""
CREATE TABLE IF NOT EXISTS procs_by_age (
    age_group   text,
    proc_count  int,
    icd9_code   text,
    short_title text,
    long_title  text,
    PRIMARY KEY ((age_group), proc_count, icd9_code)
) WITH CLUSTERING ORDER BY (proc_count DESC);
""")

In [111]:
tables = session.execute("""
  SELECT table_name 
    FROM system_schema.tables 
   WHERE keyspace_name='homework2';
""")
print([r.table_name for r in tables])

['drug_counts_by_ethnicity', 'drug_usage_by_eth', 'github', 'icu_stay_records', 'prescriptions_by_ethnicity', 'procedures_by_age_group', 'procs_by_age', 'top_drug_by_ethnicity', 'top_drug_summary', 'top_procedures_by_age', 'top_procedures_by_age_rank']


In [112]:
query_procedures = """
WITH proc_age AS (
  SELECT
    pr.icd9_code,
    -- compute age at admission in years
    date_diff('year', pa.dob, ad.admittime) AS age
  FROM procedures_icd pr
  JOIN admissions ad
    ON pr.hadm_id = ad.hadm_id
  JOIN patients pa
    ON pr.subject_id = pa.subject_id
),
buckets AS (
  SELECT
    icd9_code,
    CASE
      WHEN age <= 19 THEN '<=19'
      WHEN age BETWEEN 20 AND 49 THEN '20-49'
      WHEN age BETWEEN 50 AND 79 THEN '50-79'
      ELSE '>80'
    END AS age_group
  FROM proc_age
),
counts AS (
  SELECT
    b.age_group,
    b.icd9_code,
    COUNT(*) AS proc_count
  FROM buckets b
  GROUP BY b.age_group, b.icd9_code
),
with_names AS (
  SELECT
    c.age_group,
    c.icd9_code,
    d.short_title,
    d.long_title,
    c.proc_count
  FROM counts c
  JOIN d_icd_procedures d
    ON c.icd9_code = d.icd9_code
),
ranked AS (
  SELECT
    *,
    ROW_NUMBER() OVER (
      PARTITION BY age_group
      ORDER BY proc_count DESC
    ) AS rn
  FROM with_names
)
SELECT
  age_group,
  icd9_code,
  short_title,
  long_title,
  proc_count
FROM ranked
WHERE rn <= 3
ORDER BY age_group, proc_count DESC;
"""

procs_df = con.execute(query_procedures).df()

# insert into cassandra
insert_proc = session.prepare("""
  INSERT INTO procs_by_age
    (age_group, proc_count, icd9_code, short_title, long_title)
  VALUES (?, ?, ?, ?, ?)
""")

for r in procs_df.itertuples(index=False):
    session.execute(insert_proc, (
      r.age_group, int(r.proc_count),
      r.icd9_code, r.short_title, r.long_title
    ))

In [121]:
rows = session.execute("SELECT age_group, icd9_code, proc_count FROM procs_by_age")
for r in rows:
    print(r)

Row(age_group='20-49', icd9_code='3893', proc_count=9)
Row(age_group='20-49', icd9_code='966', proc_count=7)
Row(age_group='20-49', icd9_code='5491', proc_count=6)
Row(age_group='>80', icd9_code='3893', proc_count=20)
Row(age_group='>80', icd9_code='9904', proc_count=13)
Row(age_group='>80', icd9_code='9604', proc_count=8)
Row(age_group='<=19', icd9_code='3893', proc_count=2)
Row(age_group='<=19', icd9_code='0331', proc_count=1)
Row(age_group='<=19', icd9_code='0353', proc_count=1)
Row(age_group='50-79', icd9_code='3893', proc_count=25)
Row(age_group='50-79', icd9_code='966', proc_count=22)
Row(age_group='50-79', icd9_code='9904', proc_count=13)


## Analysis Question 3

In [118]:
session.execute("""
CREATE TABLE IF NOT EXISTS icu_los_by_gender (
    gender   text PRIMARY KEY,
    n_stays  int,
    avg_los  double,
    min_los  double,
    max_los  double
);
""")

In [119]:
gender_df = con.execute("""
  SELECT pa.gender,
         COUNT(*) AS n_stays,
         AVG(ic.los) AS avg_los,
         MIN(ic.los) AS min_los,
         MAX(ic.los) AS max_los
    FROM icustays ic
    JOIN patients pa ON ic.subject_id = pa.subject_id
   GROUP BY pa.gender;
""").df()

ins_gender = session.prepare("""
  INSERT INTO icu_los_by_gender
    (gender, n_stays, avg_los, min_los, max_los)
  VALUES (?, ?, ?, ?, ?)
""")
for r in gender_df.itertuples(index=False):
    session.execute(ins_gender, (
      r.gender, int(r.n_stays),
      float(r.avg_los), float(r.min_los), float(r.max_los)
    ))


In [120]:
rows = session.execute("SELECT * FROM icu_los_by_gender")
for r in rows:
    print(r)

Row(gender='M', avg_los=3.5138301369863, max_los=21.4136, min_los=0.1059, n_stays=73)
Row(gender='F', avg_los=5.54007142857143, max_los=35.4065, min_los=0.1904, n_stays=63)


In [123]:
session.execute("""
CREATE TABLE IF NOT EXISTS icu_los_by_ethnicity (
    ethnicity text PRIMARY KEY,
    n_stays   int,
    avg_los   double,
    min_los   double,
    max_los   double
);
""")

In [125]:
tables = session.execute("""
  SELECT table_name
    FROM system_schema.tables
   WHERE keyspace_name='homework2';
""")
print([r.table_name for r in tables])

['drug_counts_by_ethnicity', 'drug_usage_by_eth', 'github', 'icu_los_by_ethnicity', 'icu_los_by_gender', 'icu_stay_records', 'prescriptions_by_ethnicity', 'procedures_by_age_group', 'procs_by_age', 'top_drug_by_ethnicity', 'top_drug_summary', 'top_procedures_by_age', 'top_procedures_by_age_rank']


In [126]:
eth_df = con.execute("""
  SELECT ad.ethnicity,
         COUNT(*) AS n_stays,
         AVG(ic.los) AS avg_los,
         MIN(ic.los) AS min_los,
         MAX(ic.los) AS max_los
    FROM icustays ic
    JOIN admissions ad ON ic.hadm_id = ad.hadm_id
   GROUP BY ad.ethnicity;
""").df()

ins_eth = session.prepare("""
  INSERT INTO icu_los_by_ethnicity
    (ethnicity, n_stays, avg_los, min_los, max_los)
  VALUES (?, ?, ?, ?, ?)
""")
for r in eth_df.itertuples(index=False):
    session.execute(ins_eth, (
      r.ethnicity, int(r.n_stays),
      float(r.avg_los), float(r.min_los), float(r.max_los)
    ))

In [128]:
rows = session.execute("SELECT * FROM icu_los_by_ethnicity")
for r in rows:
    print(r)

Row(ethnicity='OTHER', avg_los=0.9260666666666667, max_los=1.9121, min_los=0.1059, n_stays=3)
Row(ethnicity='BLACK/AFRICAN AMERICAN', avg_los=7.676671428571429, max_los=31.1235, min_los=0.8592, n_stays=7)
Row(ethnicity='WHITE', avg_los=4.130488043478262, max_los=35.4065, min_los=0.1904, n_stays=92)
Row(ethnicity='ASIAN', avg_los=3.89005, max_los=7.1173, min_los=0.6628, n_stays=2)
Row(ethnicity='HISPANIC/LATINO - PUERTO RICAN', avg_los=3.2430666666666665, max_los=10.8625, min_los=0.7569, n_stays=15)
Row(ethnicity='UNKNOWN/NOT SPECIFIED', avg_los=4.925272727272727, max_los=15.041, min_los=1.6444, n_stays=11)
Row(ethnicity='UNABLE TO OBTAIN', avg_los=13.357, max_los=13.357, min_los=13.357, n_stays=1)
Row(ethnicity='AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE', avg_los=11.33715, max_los=21.4136, min_los=1.2607, n_stays=2)
Row(ethnicity='HISPANIC OR LATINO', avg_los=7.459633333333333, max_los=15.0564, min_los=3.5465, n_stays=3)
